
# Lecture 2 — Numerical Optimization Basics in DICE (Python)

This notebook complements the slides of the **Lecture 2** on carbon pricing and optimal control. We move from scenario-building (*what could happen*) to the social planner's problem (*what should happen*), and show how the **optimal carbon price (SCC)** emerges and is computed numerically.

**Roadmap**  
1. Load the model and inspect the state/control structure.  
2. Restate the planner's **objective** and time truncation (finite-horizon approximation).  
3. Solve numerically with a rolling planning window; **read SCC** from outputs.  
4. Sensitivity (e.g., higher damages), and exporting results.



## 1. Setup
We import the DICE model components and ensure Python can find the module.


In [ ]:

import os, sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Make sure we can import the user's module
sys.path.append('/mnt/data')
from importlib import reload
import DICE
reload(DICE)

from DICE import Params, init_states, update_path, obj_fun, run_optimal_policy, mat_to_df
print("DICE loaded from:", DICE.__file__)



## 2. Model skeleton (states & controls)

- **States (examples):** capital $K_t$, TFP $A_t$, population $L_t$, climate stocks $M_{\mathrm{AT}}, M_{\mathrm{UP}}, M_{\mathrm{LO}}$, temperatures $T_{\mathrm{AT}}, T_{\mathrm{LO}}$.
- **Controls:** saving rate $s_t$, abatement rate $\mu_t$ (which, together with technology parameters, implies a **carbon price / tax** path).
- **Units:** $C_t$ in trillion USD, $L_t$ in million → per-capita consumption in **thousand USD**: $c_t = 1000 \times C_t/L_t$.


In [ ]:

# Instantiate parameters and construct an initial path
p = Params()
sim = init_states(p)

# Baseline controls for initialization (not optimal): small abatement, constant saving
sim[:, p.i_mu] = 0.03
sim[:, p.i_s]  = 0.20

# Update path for all endogenous variables given the controls
timevec = range(1, p.nT)
sim = update_path(sim, timevec, p)

# Peek at available columns
print("Columns:", p.col)
print("Horizon length (periods):", p.nT, " — years:", sim[-1, p.i_time])



## 3. Planner's objective (CRRA utility)

We maximize discounted social welfare
$$
W = \sum_{i=0}^{I^\star} \beta_\Delta^{\,i}\, L_{t+i\Delta}\, u\!\left( 1000\,\frac{C_{t+i\Delta}}{L_{t+i\Delta}} \right), 
\qquad \beta_\Delta=(1+\rho)^{-\Delta},
$$
with CRRA utility $u(c)=\frac{c^{1-\gamma}-1}{1-\gamma}$. The notebook uses the `obj_fun` provided in the module.

**Note on truncation:** We replace the infinite horizon with a finite $I^\star$ chosen so the tail weight is below a tolerance.


In [ ]:

# Examine the objective on a constant-control path (sanity check)
# Here we 'flatten' a control (e.g., saving only) and evaluate welfare.
x_const = np.full(p.nT-1, 0.20)  # constant saving
W_val = obj_fun(x_const, sim, timevec, p, [p.i_s])
print("Objective value (negative welfare) on constant s=0.20 path:", W_val)



## 4. Finite-horizon planning window $I^{\star}$

We select a planning window so that the discount weight of the last term is below the numerical tolerance `p.toly`. This implements the finite-horizon approximation discussed in the slides.


In [ ]:

# Compute planning window similar to DICE.run_optimal_policy
disc, Tplanner = 1.0, 1
while disc > p.toly:
    Tplanner += 1
    disc *= (1.0/(1.0 + p.rho))**p.Delta

print("Chosen planning window (periods):", Tplanner, " (each period = Δ years =", p.Delta, ")")



## 5. Solve a first problem: **savings only** (no transition control)

We optimize only the saving rate $s_t$ (bounds from the parameter file), keeping $\mu_t$ fixed. This provides a baseline "no-transition" benchmark. The problem reads as:

$$
\begin{aligned}
\max_{\{s_{t+i}\}_{i=0}^{I^\star}} \quad 
& \sum_{i=0}^{I^\star} \beta^{i}\, L_{t+i\Delta}\;
u\!\left( 1000\,\frac{C_{t+i\Delta}}{L_{t+i\Delta}} \right) \\[0.4em]
\text{s.t.}\quad 
& y_{t+i} \;=\; G_p\!\big(y_{t+i-1},\, x_{t+i-1},\, s_{t+i},\, \bar{\mu}_{t+i};\big), 
\quad i=0,\dots,I^\star\!-\!1,\\
& \mu_{t+i} \;=\; \bar{\mu}_{t+i}\ \ (\text{fixed}),\\
& s_{\min} \;\le\; s_{t+i} \;\le\; s_{\max}\ \ (\text{box constraints}),\\
& y_{t-1} \;=\; y^{\mathrm{given}}_{t-1},
\end{aligned}
$$

with $u(c)=\dfrac{c^{1-\gamma}-1}{1-\gamma}$, $\gamma>0$, and $\beta=(1+\rho)^{-\Delta}$.
Here $C_t$ is in trillion USD and $L_t$ in million, so $c_t = 1000\,C_t/L_t$ is in thousand USD per capita.  
**Bounds in code:** $s_{\min}=$`p.s_lower`, $s_{\max}=$`p.s_upper`; $\mu_t$ fixed at the preset path (e.g. `path[:, p.i_mu]`).



In [ ]:

print("Solving: savings-only optimal path...")
bounds_s   = (p.s_lower, p.s_upper)
control_id = [p.i_s]
path_opt_s = run_optimal_policy(sim.copy(), timevec, p, bounds_s, control_id)

print("Done. Preview of s_t path (first 5):", path_opt_s[:5, p.i_s])



## 6. Solve the full problem: **savings + abatement**

Now we optimize the pair $(s_t, \mu_t)$. The carbon tax path is then implied by the model and reported in the `Tax` column. The problem reads as:

$$
\begin{aligned}
\max_{\{s_{t+i},\,\{\mu_{t+i}\}_{i=0}^{I^\star}} \quad 
& \sum_{i=0}^{I^\star} \beta^{i}\, L_{t+i\Delta}\;
u\!\left( 1000\,\frac{C_{t+i\Delta}}{L_{t+i\Delta}} \right) \\[0.4em]
\text{s.t.}\quad 
& y_{t+i} \;=\; G_p\!\big(y_{t+i-1},\, x_{t+i-1},\, s_{t+i},\, \mu_{t+i};\big), 
\quad i=0,\dots,I^\star\!-\!1,\\
& s_{\min} \;\le\; s_{t+i} \;\le\; s_{\max}\ \ (\text{box constraints}),\\
& 0 \;\le\; \mu_{t+i} \;\le\; 1\ \ (\text{box constraints}),\\
& y_{t-1} \;=\; y^{\mathrm{given}}_{t-1},
\end{aligned}
$$


In [ ]:

print("Solving: joint (s_t, μ_t) optimal path...")
bounds_smu   = [(p.s_lower, p.s_upper), (0.0, 1.0)]
control_id   = [p.i_s, p.i_mu]
path_opt_smu = run_optimal_policy(sim.copy(), timevec, p, bounds_smu, control_id)

print("Done. Preview of μ_t path (first 5):", path_opt_smu[:5, p.i_mu])



## 7. Visualize the optimal controls and the implied carbon price

Below we plot (one chart per figure) the optimal saving \(s_t\), abatement \(\mu_t\), and the implied carbon tax (USD/tCO\(_2\)) reported in `Tax`.


In [ ]:

# Plot saving rate
plt.figure(figsize=(6,3.8))
plt.plot(path_opt_smu[:, p.i_time], path_opt_smu[:, p.i_s], label="Optimal transition (s_t)")
plt.plot(path_opt_s[:,  p.i_time], path_opt_s[:,  p.i_s],  linestyle="--", label="No transition (s_t)")
plt.xlabel("Year"); plt.ylabel("Saving rate s_t"); plt.title("Optimal saving rate")
plt.legend(); plt.grid(True); plt.tight_layout()


In [ ]:

# Plot abatement rate
plt.figure(figsize=(6,3.8))
plt.plot(path_opt_smu[:,p.i_time], path_opt_smu[:,p.i_mu], label="Optimal transition (μ_t)")
plt.xlabel("Year"); plt.ylabel("Abatement rate μ_t"); plt.title("Optimal abatement rate")
plt.legend(); plt.grid(True); plt.tight_layout()


In [ ]:

# Plot implied carbon tax (USD per tCO2)
plt.figure(figsize=(6,3.8))
plt.plot(path_opt_smu[:, p.i_time], path_opt_smu[:, p.i_Tax], label="Implied carbon tax (USD/tCO₂)")
plt.xlabel("Year"); plt.ylabel("Carbon tax (USD/tCO₂)"); plt.title("Optimal carbon price / SCC (model-implied)")
plt.legend(); plt.grid(True); plt.tight_layout()



## 8. Export results

Convert arrays to a tidy `DataFrame` for further analysis or plotting elsewhere.


In [ ]:

df_baseline = pd.DataFrame({
    "year": path_opt_smu[:, p.i_time],
    "s":    path_opt_smu[:, p.i_s],
    "mu":   path_opt_smu[:, p.i_mu],
    "tax":  path_opt_smu[:, p.i_Tax],
    "E":    path_opt_smu[:, p.i_E],
    "T_AT": path_opt_smu[:, p.i_T_AT],
    "C":    path_opt_smu[:, p.i_C],
})
df_baseline.head()



### Wrap-up

- The **optimal carbon price** reported in `Tax` provides the model's normative guidance (SCC).  
- The **finite-horizon window** implements the truncation discussed in the lecture.  
- The **rolling solve** produces full time paths for states and controls.  
- Sensitivity experiments (e.g., damages) directly shift \(\mu_t\) and the implied tax path.
